In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import os

# 結算流程

| 按讚模式(函式)         | 留言模式(函式)         |        資料表     |   
|-----------------|------------------|----------------|
|表單修改數量|FB留言處修改數量|orginalBigSheet
|匯入團購規則|匯入團購規則|settlementRule、deliverRule、deliverFreeAllRule、innoproVIP(留言模式無)|
|限加一過濾(orderSort)| 限加一過濾(limitProcess)      | limitProcess
|量多優先排序(orderSort)|      --          |limitProcess
|VIP排序(vipMove)| -- |limitProcess
|購買權確認(confirmPurchaseRight)|購買權確認(confirmPurchaseRight)|confirmPurchaseRight
|買X送Y計價(confirmPurchaseRight)|買X送Y計價(confirmPurchaseRight)| confirmPurchaseRight
|X瓶以上免運(confirmPurchaseRight)|X瓶以上免運(confirmPurchaseRight)|confirmPurchaseRight
|各類產品運費計算(calDeliverRate、calDeliverCost)|各類產品運費計算(calDeliverRate、calDeliverCost)|calDeliverCost
|全團優惠(calDeliverCostFinal)|全團優惠(calDeliverCostFinal)|calDeliverCostFinal
|確認清單(afterSettleList)|確認清單(afterSettleList)|afterSettleList
|結算金額資訊(feeList)|結算金額資訊(feeList)|feeList

# 創資料庫

In [2]:
# def createSQLDatabase(newDatabaseName):
#     db = pymysql.connect(host='localhost', port=3306, user='root', passwd='benbilly3@', charset='utf8')
#     cursorInsatnce  =db.cursor()  
#     # SQL Statement to create a database
#     sqlStatement = "CREATE DATABASE "+newDatabaseName  
#     # Execute the create database SQL statment through the cursor instance
#     cursorInsatnce.execute(sqlStatement)
# createSQLDatabase('settleProcessTdaFast')

# 資料庫連線

In [23]:
connect_info = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format('root','benbilly3@','localhost',3306,'settleProcess')  #1
engine = create_engine(connect_info)
db = pymysql.connect('localhost', 'root', 'benbilly3@','settleProcess', charset='utf8')


((749,
  '97900100720300',
  datetime.date(2019, 7, 31),
  datetime.timedelta(0, 56164),
  '00449',
  '8220000082540329133 ',
  3005,
  'CD',
  '',
  None),
 (750,
  '97900100720300',
  datetime.date(2019, 7, 31),
  datetime.timedelta(0, 62701),
  '00451',
  '8220000875540300550 ',
  5460,
  'CD',
  '',
  None),
 (751,
  '97900100720300',
  datetime.date(2019, 7, 31),
  datetime.timedelta(0, 58861),
  '00450',
  '0130000212502065120 ',
  1700,
  'CD',
  '',
  None),
 (752,
  '97900100720300',
  datetime.date(2019, 7, 31),
  datetime.timedelta(0, 63440),
  '00452',
  '0080000252200078188 ',
  12430,
  'CD',
  '',
  None),
 (753,
  '97900100720300',
  datetime.date(2019, 7, 31),
  datetime.timedelta(0, 67319),
  '00453',
  '0120000754168070017 ',
  9640,
  'CD轉',
  '',
  None),
 (754,
  '97900100720300',
  datetime.date(2019, 7, 31),
  datetime.timedelta(0, 67975),
  '00454',
  '7000002811710118286 ',
  4925,
  'CD轉',
  '',
  None),
 (755,
  '97900100720300',
  datetime.date(2019, 7, 31)

# 團次控制

In [4]:
session='20190911'

# 匯入團購規則

1.留言數量登記

2.品項屬性優惠

3.運費規則

全4.館優惠

5.VIP名單

In [8]:
def orginalBigSheet(session,fileUrl):
    cursor=db.cursor()
    sql="DELETE FROM ORIGINALBIGSHEET WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
    df=pd.read_csv(fileUrl)
    try:
        df=df.rename(columns={'amount':'orderCount','member__fbid':'fbuid',
                      'member__name':'username','product__wine_id':'wineID',
                       'session_id':'session'      })
        df=df.loc[:,['session','wineID','comment_order','username','fbuid','orderCount']]
    except:
        pass

    df.to_sql(name='orginalBigSheet', con=engine, if_exists='append')
    return df

orginalBigSheet(session,'/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/20190911/20190911_final_orders.csv')

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,session,wineID,comment_order,username,fbuid,orderCount
0,20190911,4473,NaN,李宏澤,100001959157859,3
1,20190911,4520,NaN,李宏澤,100001959157859,3
2,20190911,4525,NaN,李宏澤,100001959157859,6
3,20190911,4667,NaN,李宏澤,100001959157859,6
4,20190911,4669,NaN,李宏澤,100001959157859,1
5,20190911,4670,NaN,李宏澤,100001959157859,1
6,20190911,4672,NaN,李宏澤,100001959157859,3
7,20190911,4473,NaN,CY Ng,1697332988,1
8,20190911,4520,NaN,CY Ng,1697332988,1
9,20190911,4526,NaN,CY Ng,1697332988,1


In [ ]:
#settlementRule
def settlementRule(session,fileUrl):
    cursor=db.cursor()
    sql="DELETE FROM SETTLEMENTRULE WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
    df=pd.read_csv(fileUrl)
    df.to_sql(name='settlementRule', con=engine, if_exists='append')
    return df
    
settlementRule(session,'/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/20190911/T大-2019-9團規則.xlsx - settlementRule.csv')

In [ ]:
#deliverRule
def deliverRule(session,fileUrl):
    cursor=db.cursor()
    sql="DELETE FROM DELIVERRULE WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
    df=pd.read_csv(fileUrl)
    df.to_sql(name='deliverRule', con=engine, if_exists='append')
    return df

deliverRule(session,'/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/20190911/T大-2019-9團規則.xlsx - deliverRule.csv')

In [ ]:
#deliverFreeAllRule
def deliverFreeAllRule(session,fileUrl):
    cursor=db.cursor()
    sql="DELETE FROM DELIVERFREEALLRULE WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
    df=pd.read_csv(fileUrl)
    df.to_sql(name='deliverFreeAllRule', con=engine, if_exists='append')
    return df

deliverFreeAllRule(session,'/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/20190911/T大-2019-9團規則.xlsx - deliverFreeAllRule.csv')

In [ ]:
#VIP_List

def innoproVIP(session,fileUrl):
    cursor=db.cursor()
    sql="DELETE FROM INNOPROVIP WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
    df=pd.read_csv(fileUrl)
    df.to_sql(name='innoproVIP', con=engine, if_exists='append')
    return df

innoproVIP(session,'/Users/benbilly3/Desktop/Innopro-master/客戶管理資訊/innoproVIP.csv')


# 結算測試

## 處理限定+1商品,訂購數大於限制數且限制數大於0

### 重要欄位說明

orginalOrderCount=原請求數

orderCount=限加處理後瓶數


In [ ]:
def limitProcess(session,method='append'):
    #刪掉重複SESSION做update
    cursor=db.cursor()
    sql="DELETE FROM lIMITPROCESS WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    df=pd.read_sql('select session,wineID,comment_order,username,fbuid,orderCount from orginalBigSheet where session='+str(session),con=engine,index_col=['wineID'])
    df.index=df.index.astype(str)
    df2=pd.read_sql('select * from settlementRule where session='+str(session),con=engine,index_col=['wineID'])
    df2.index=df2.index.astype(str)
    for i in ['price','stockNum','wineName','style','deliverFree','sale','limit']:
        df[i]=df2[i]
    df['orginalOrderCount']=df['orderCount']
    #處理限定+1商品,訂購數大於限制數且限制數大於0
    df['orderCount']=[b if (b>0)&(a>b) else a for a,b in zip(df['orderCount'],df['limit'])]
    df=df.reset_index()#解索引才能存SQL
    df.to_sql(name='limitProcess', con=engine, if_exists=method)#if_exists='append 可疊加不同資料
    return df
a=limitProcess(session)
a


# 量多優先排序

In [9]:
def orderSort(session):

    df=pd.read_sql('select session,wineID,comment_order,username,fbuid,orderCount from orginalBigSheet where session='+str(session),con=engine,index_col=['wineID'])
    df.index=df.index.astype(str)
    df2=pd.read_sql('select * from settlementRule where session='+str(session),con=engine,index_col=['wineID'])
    df2.index=df2.index.astype(str)
    for i in ['price','stockNum','wineName','style','deliverFree','sale','limit']:
        df[i]=df2[i]
    df['orginalOrderCount']=df['orderCount']
    #先處理限加一，排序才公平，因為有人留言會亂留
    df['orderCount']=[b if (b>0)&(a>b) else a for a,b in zip(df['orderCount'],df['limit'])]
    df3=df.reset_index()#解決series問題
    dataset=[]
    for i in list(dict.fromkeys(df.index)):
        df2=df3[df3['wineID']==i]
        df2=df2.sort_values('orderCount',ascending=False)
        df2['comment_order']=[a for a in range(0,len(df2))]
        dataset.append(df2)
    orderSort=pd.concat(dataset)
    orderSort=orderSort.set_index('wineID')
    return orderSort
orderSort(session)

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,session,comment_order,username,fbuid,orderCount,price,stockNum,wineName,style,deliverFree,sale,limit,orginalOrderCount
wineID,,,,,,,,,,,,,
4473,20190911,0,李宏祥,1288033764,18,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,18
4473,20190911,1,522,100026354479057,6,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,6
4473,20190911,2,Leon Chen,1847646753,6,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,6
4473,20190911,3,Carol Wu,100004278498557,4,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,4
4473,20190911,4,Andy Yu,100002606253012,4,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,4
4473,20190911,5,Leon Chen,545479715,4,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,4
4473,20190911,6,Paul Wang,1495955578,4,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,4
4473,20190911,7,356,100016231086008,4,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,4
4473,20190911,8,蔡簣帆,1188113560,4,680,412,Tenuta San Guido 2015 le Difese紅,0,0,0,0,4


# VIP排序

In [31]:
def vipMove(session):
    
    #刪掉重複SESSION做update
    cursor=db.cursor()
    sql="DELETE FROM lIMITPROCESS WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    df2=orderSort(session)
    vip=pd.read_sql('select VIPfbuid from innoproVIP where session='+str(session),con=engine)
    vip=vip['VIPfbuid'].values
    
    #取出庫存數與限加一數量
    stock=pd.read_sql('select wineID,stockNum from settlementRule where session='+str(session),con=engine,index_col=['wineID'])
    stock.index=stock.index.astype(str)
    orderCountSum=df2.groupby('wineID')['orderCount'].sum()
    stock['orderCountSum']=orderCountSum
    vipActList=stock[stock['orderCountSum']>stock['stockNum']]#庫存<購買總量才需帶入VIP優先權調整
    
    data=[]
    for i,n in zip(vipActList.index,vipActList['stockNum']):
        df3=df2.loc[i]
        
        #若購買數大於庫存數則進行VIP優先權上移調整
        df3['orderCountCumsum']=df3['orderCount'].cumsum()
        safe=df3[df3['orderCountCumsum']<=n]#安全名單
        out=df3[df3['orderCountCumsum']>n]#摃龜名單
        df3=df3[df3['orderCount']>0]#拿掉訂購<1的訂單
        
        #safe再分購買數大於或小於等於最小值
        minN1=min(list(dict.fromkeys(safe['orderCount'])))
        select1=safe[safe['orderCount']>=minN1]
        select2=safe[safe['orderCount']<minN1]

        VIP_Move=[]
        for v in vip :
            if v in out['fbuid'].values.tolist():
                df=out[out['fbuid']==v]
                VIP_Move.append(df)
        if len(VIP_Move)>0:
            print('VIP in out',i)
            VIP_Move2=pd.concat(VIP_Move)
        else:
            print('VIP not in out',i)
            VIP_Move2=out#若沒有vip，則輸出out
        if max(list(dict.fromkeys(safe['orderCount'])))==1: 
            final=pd.concat([VIP_Move2,select1,select2,out])
        else:
            final=pd.concat([select1,VIP_Move2,select2,out])
        final=final[~final.duplicated(keep='first')]#將VIP從out內移除
        final['comment_order']=[a for a in range(0,len(final))]#重新排序
        final=final.drop(columns='orderCountCumsum').reset_index()
        data.append(final)
    up=pd.concat(data)
    down=df2.drop(index=list(vipActList.index)).reset_index()
    vipMove=pd.concat([up,down])
#     vipMove.to_sql(name='limitProcess', con=engine, if_exists='append')
    return vipMove
vipMove(session)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


VIP in out 4609
VIP in out 4610
VIP in out 4871
VIP in out 4592
VIP not in out 4527
VIP in out 4526
VIP in out 4520
VIP in out 4525
VIP in out 4490
VIP in out 4521
VIP not in out 4412
VIP in out 4479
VIP in out 4534


,wineID,session,comment_order,username,fbuid,orderCount,price,stockNum,wineName,style,deliverFree,sale,limit,orginalOrderCount
0,4609,20190911,0,洪思群,100000090961284,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
1,4609,20190911,1,Bai Xiao Bai,643760847,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
2,4609,20190911,2,Frank Fang,1520874579,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
3,4609,20190911,3,Herb Chang,580856401,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
4,4609,20190911,4,曾彥文,100001106450487,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
5,4609,20190911,5,Da Chen Pang,100007086393208,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
6,4609,20190911,6,Alex Chun-Wei Kuo Chang,1402197858,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
7,4609,20190911,7,Yi-Hung Pan,100000332146403,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
8,4609,20190911,8,Po-Yu Chen,680673028,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1
9,4609,20190911,9,Greg Liu,100000203217938,1,1250,6,Benedicte et Stephane Tissot 2016 Arbois Chard...,0,0,0,0,1


# confirmList

### 決定購買權，確認購買清單，處理買Ｘ送Ｙ優惠

orderCountCumsum=訂購累加數

getSure=顯示有無買到(sorry為沒買到,get為買到)

totalPrice=個人單項商品總價

orderCount_sum=單項商品總訂購數

In [ ]:
def confirmPurchaseRight(session,method='append'):
    #刪掉重複SESSION做update
    cursor=db.cursor()
    sql="DELETE FROM CONFIRMPURCHASERIGHT WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    df=pd.read_sql('select * from limitProcess where session='+str(session),con=engine,index_col=['wineID'])
    d=[]
    for wineID in list(dict.fromkeys(df.index)):
        try:
            if df.index.tolist().count(wineID)<2:#只有一篇留言的單獨抓出,因為是series要獨自處理
                t=df.iloc[df.index.tolist().index(wineID):df.index.tolist().index(wineID)+1]#Series轉dataframe
            else:
                t=df.loc[wineID]
                t=t.sort_values(["comment_order"])#留言排序
            t['orderCountCumsum']=t['orderCount'].cumsum()#累加數量
            cond1=t['orderCountCumsum']<=t['stockNum']#設定安全名單條件
            t_in=t[cond1]#安全名單
            t_out=t[~cond1]#排除名單
            #最後1人order的數量大於剩餘庫存(t_out名單第一人)，則電腦自動更新其最大可買數
            if ((t_in['stockNum']-t_in['orderCountCumsum']).iloc[-1]>0) and len(t_out)>0:#安全名單未滿足庫存,且還有人在排隊時
                t_out['orderCount'].iloc[0]=((t_in['stockNum']-t_in['orderCountCumsum']).iloc[-1])#調整最後一人數量
                final=pd.concat([t_in,t_out.iloc[:1]])#把最後一人再加回
                t_out=t_out.iloc[1:]
                final=pd.concat([final,t_out])#名單重整合併
            else:
                final=pd.concat([t_in,t_out])#名單重整合併
        except:
            print('error',wineID)#錯誤回報
            pass
        final['orderCountCumsum']=final['orderCount'].cumsum()#重新加總購買量
        s=final['orderCountCumsum']-final['stockNum']#顯示有無無買到
        final['getSure']=['get' if i<=0 else 'sorry' for i in s]#顯示有無無買到

        d.append(final)
    result=pd.concat(d)
    
    #提取買Ｘ送Ｙ規則資料來計價
    result['saleBasic']=result['sale'].apply(lambda s:pd.to_numeric(s[:-2]) if str(s)!='0' else 1)#買Ｘ,else=1，讓無優惠品項製造基底為1
    result['saleNum']=result['sale'].apply(lambda s:pd.to_numeric(s[-1]) if str(s)!='0' else 0)#送Ｙ,else=，，讓無優惠品項歸0
    #計算贈送瓶數
    result['giftBottle']=result['orderCount']//(result['saleBasic']+result['saleNum'])*result['saleNum']
    result['giftBottle']=[0 if a==b else a for a,b in zip(result['giftBottle'],result['orderCount'])]
    #計算計價基準數量
    result['orderCountAfterSale']=result['orderCount']-result['giftBottle']#原瓶數除優惠基底*加送變數+原瓶數=優惠後訂購數
    #sorry 的數量調成0
    result['orderCount']=[0 if a=='sorry'else b for a,b in zip(result['getSure'],result['orderCount'])]
    result['orderCountAfterSale']=[0 if a=='sorry'else b for a,b in zip(result['getSure'],result['orderCountAfterSale'])]
    
    #計算確認單品總價，收尾處理(未來若要擴充成箱價在這裡處理)
    result['totalPrice']=result['orderCountAfterSale']*result['price']

    result['orderCount_sum']=result.groupby(['wineID'])['orderCount'].sum()#商品銷售總量
    result= result[result['orginalOrderCount']!=0]#刪掉留言為0的
    result=result.reset_index()
    if 'index' in result.columns.values:
        result=result.drop(columns=['index'])
    result.to_sql(name='confirmPurchaseRight', con=engine, if_exists=method)
        
    return result


confirmPurchaseRight(session)

# 組合價處理

暫時不用

In [ ]:
#     #組合價處理
#     if 'C1' in result['combination'].values:
#         result['combination']=result['combination'].astype(str)
#         conb=result[result['combination']!='0']
#         no_conb=result[result['combination']=='0']#非組合商品
#         conb=conb.reset_index()
#         conb=conb.set_index(['fbuid','combination'])

#         #把sorry拿掉,因為數量為0
#         conb_sorry=conb[conb['getSure']=='sorry']
#         conb=conb[conb['getSure']=='get']

#         #計算買Ｘ送Ｙ後的訂購數

#         conb['conbSaleBasic']=conb['combinationSale'].apply(lambda s:pd.to_numeric(s[:-2]) if str(s)!='0' else 1)#買Ｘ,else=1，讓無優惠品項製造基底為1
#         conb['conbSaleNum']=conb['combinationSale'].apply(lambda s:pd.to_numeric(s[-1]) if str(s)!='0' else 0)#送Ｙ,else=，，讓無優惠品項歸0
#         conbBottleSum=conb.groupby(['fbuid','combination'])['orderCount'].sum()
#         conb['conbBottleSum']=conbBottleSum
#         conb['conbBottlegift']=conb['conbBottleSum']//(conb['conbSaleBasic']+conb['conbSaleNum'])*conb['conbSaleNum']
#         conbCustomerList=conb.groupby(['combination','fbuid','wineID'])['orderCount','conbBottlegift','orderCountAfterSale','price'].sum()

#         #組合品的商品清單
#         conbCommodityList = list(dict.fromkeys(conbCustomerList.index.get_level_values(0)))

#         #贈送量先拿掉低價品

#         conb_dataset=[]#個別商品單獨處理
#         for c in conbCommodityList:
#             test=conbCustomerList.loc[c]
#             test['combination']=c
#             nextList=test
#             #組合品的會員清單
#             conbMemberList = list(dict.fromkeys(nextList.index.get_level_values(0)))
#             for i in conbMemberList :#個別客戶單獨處理
#                 test=nextList.loc[i].sort_values('price')
#                 test['fbuid']=i
#                 test['conbBottlegift'].loc[1:]=None#
#                 test['orderCountCumsum']=test['orderCount'].cumsum()
#                 test['cut']=test['conbBottlegift']-test['orderCountCumsum']
#                 test['orderCountAfterSale']=[0 if (a-b>=0)&(c>=0) else b-a if (b-a>0)&(c<0) else b for a,b,c in zip(test['conbBottlegift'],test['orderCountAfterSale'],test['cut'])]
#                 test['cut']=test['cut'].shift(1)
#                 test['orderCountAfterSale']=[b-c if (b>=c)&(c>=0) else b for a,b,c in zip(test['conbBottlegift'],test['orderCountAfterSale'],test['cut'])]
#                 test['fbuid']=i
#                 test=test.reset_index()
#                 test=test.set_index(['combination','fbuid','wineID'])
#                 conb_dataset.append(test)
#         conb_dataset=pd.concat(conb_dataset)

#         #算組合商品各品項贈送量
#         conb_dataset['condGiftNum']=conb_dataset['orderCount']-conb_dataset['orderCountAfterSale']

#         #組裝資料
#         conb=conb.reset_index()
#         conb=conb.set_index(['combination','fbuid','wineID'])
#         conb_sorry=conb_sorry.reset_index()
#         conb_sorry=conb_sorry.set_index(['combination','fbuid','wineID'])
#         conb['orderCountAfterSale']=conb_dataset['orderCountAfterSale']
#         conb['giftBottle']=conb_dataset['condGiftNum']
#         conb=pd.concat([conb,conb_sorry])
#         conb['totalPrice']=conb['price']*conb['orderCountAfterSale']
#         conb=conb.sort_index()
#         conb['conbBottlegift']=conb['conbBottlegift'].fillna(method='ffill')
#         conb=conb.reset_index()
#         conb=conb.set_index(['wineID'])
#         result=pd.concat([no_conb,conb])
#         result=result.drop(columns=['conbBottleSum','conbBottlegift','conbSaleBasic','conbSaleNum'])

# 運費規則計算函式

In [ ]:
def calDiliverRate(num,n1,p1,n2,p2,n3,p3):
    if n1==0:#若n1為0則該類別全免運
        d_p=num*0
    else:
        if num==n1:
            d_p=p1
        else:
            if n1<num<=n2:
                d_p=p2
            elif n2<num<=n3:
                d_p=p3
            else:
                q=num//n3
                r=num%n3
                if 0<r<=n2:
                    d_p=p2+q*p3
                elif r==0:
                    d_p=q*p3
                else:
                    d_p=(q+1)*p3
    return d_p

calDiliverRate(30,1,160,6,225,12,290)

# 第一階段運費計算

各類商品分開算運費

In [ ]:
def calDeliverCost(session,method='append'):#
    #刪掉重複SESSION做update
    cursor=db.cursor()
    sql="DELETE FROM CALDILIVERCOST WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
    
    df=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['wineID'])
    df['session']=df['session'].astype(str)
    
    #只納入有購買權的
    df=df[df['getSure']=='get']
    #納入單品項運費優惠計算(X瓶以上免運)
    df['orderCountAfterDiliverFree']= [0 if (a>0)&(b>=a) else b for a,b in zip(df['deliverFree'],df['orderCount'])]


    #依照個人ID、團次、商品屬性計算運費和總價
    df=df.groupby(['fbuid','session','style'])['orderCount','orderCountAfterDiliverFree','totalPrice'].sum()
    df=df.reset_index()
    df['session']=df['session'].astype(str)
    df['style']=df['style'].astype(str)
    df=df.set_index(['session','style'])
    
    #引入運費規則表
    deilverRule=pd.read_sql('select * from deliverRule where session='+str(session),con=engine)
    deilverRule['session']=deilverRule['session'].astype(str)
    deilverRule['style']=deilverRule['style'].astype(str)
    deilverRule=deilverRule.set_index(['session','style'])
    
    for i in deilverRule.columns[-6:]:
        df[i]=deilverRule[i]
        
    #計算各類商品運費
    df['diliverCost']=[calDiliverRate(i,d1,p1,d2,p2,d3,p3)
                           for i,d1,p1,d2,p2,d3,p3 in zip(
                                        df['orderCountAfterDiliverFree'],
                                        df['deliverRule1'],df['deliverRule1Price'],
                                        df['deliverRule2'],df['deliverRule2Price'],
                                        df['deliverRule3'],df['deliverRule3Price']                           
                                                                 )]
    #自取總價
    df['selfGetCost']=df['totalPrice']
    df=df.reset_index()#解索引才能存SQL
    df.to_sql(name='calDeliverCost', con=engine, if_exists=method)#if_exists='append 可疊加不同資料
    return df

calDiliverCost(session)

# 全館優惠最終結算

## 各類商品運費合併

selfGetCost=折扣前總價

diliverCostAfterDiscountAll=全館折扣後總價(自取價)

diliverCostAfterDeilverFreeAll=最終運費

totalPriceIncludeDC=最終含運總價

In [ ]:
def calDiliverCostFinal(session,method='append'):
    #刪掉重複SESSION做update
    cursor=db.cursor()
    sql="DELETE FROM CAlDELIVERCOSTFINAL WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
    
    df=pd.read_sql('select * from calDeliverCost where session='+str(session),con=engine)
    df['session']=df['session'].astype(str)
    df=df.set_index(['session','style'])
    
    df=df.groupby(['session','fbuid'])['orderCount','selfGetCost','diliverCost'].sum()
    df=df.reset_index()
    df=df.set_index(['session'])
    #引入運費規則表
    Rule=pd.read_sql('select * from deliverFreeAllRule where session='+str(session),con=engine)
    Rule['session']=Rule['session'].astype(str)
    Rule=Rule.set_index('session') 
    for i in Rule.columns[:]:
        df[i]=Rule[i]
        
    #全館折扣
    df['diliverCostAfterDiscountAll']=df['selfGetCost']*df['discountAll']
    #全館免運優惠，若買進清單同時有單品運費優惠，則先以全館免運為優先，不適用的話，再用原單品優惠
    df['diliverCostAfterDeilverFreeAll']=[0 if ((a>=e)&(e>0))or((b>=d)&(d>0)) else c
                                          for a,b,c,d,e in zip(df['orderCount'],df['diliverCostAfterDiscountAll'],
                                          df['diliverCost'],df['deilverFreeAll'],df['deilverFreeAllbottle'])]
                                 
#     千辛萬苦弄出的總價
    df['totalPriceIncludeDC']=df['diliverCostAfterDiscountAll']+df['diliverCostAfterDeilverFreeAll']
    df=df.reset_index()#解索引才能存SQL
    if 'index' in df.columns.values:
        df=df.drop(columns=['index'])
    df.to_sql(name='calDeliverCostFinal', con=engine, if_exists=method)#if_exists='append 可疊加不同資料

    return df

calDiliverCostFinal(session)

# 確認清單

In [11]:
def afterSettleList(session,method='append'):
    cursor=db.cursor()
    sql="DELETE FROM AFTERSETTLELIST WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    df=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['session','fbuid'])

    dfx=df.loc[:,['username','getSure','wineID','wineName','limit','orginalOrderCount','orderCount','giftBottle','orderCountAfterSale',
                 'price','totalPrice']]
    dfx['limit']=dfx['limit'].apply(lambda s:"限加"+str(s) if s>0 else '無限加限制')
    df_sorry=dfx[dfx['getSure']=='sorry']
    df_sorry['orderCountAfterSale']=0
    df_sorry['totalPrice']=0

    df_get=dfx[dfx['getSure']=='get']

    final_list=pd.concat([df_get,df_sorry])
    final_list=final_list.rename(columns={'username':'FB姓名','getSure':'購買權確認','wineID':'商品編號','wineName':'商品名稱',
                          'limit':'購買限制','orginalOrderCount':'留言購買數','orderCount':'結算確認購買數量','giftBottle':'優惠贈送量',
                         'orderCountAfterSale':'扣除贈送量後計價數','price':'商品單價','totalPrice':'商品總價'})
    final_list['備註']=['未排到' if (a=='sorry') 
                      else b if (a=='get')&(b!='無限加限制')&(c>0)
                      else '留言購買數大於庫存數,修正為最大可買數。' if (a=='get')&(b=='無限加限制')&(c>d) 
                      else '已確認'
                      for a,b,c,d in zip(final_list['購買權確認'],final_list['購買限制'],final_list['留言購買數'],final_list['結算確認購買數量'])]
    final_list=final_list.reset_index()
    final_list.to_sql(name='afterSettleList', con=engine, if_exists=method)

    return final_list.sort_index()

a=afterSettleList(session)
a.to_csv('/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/20190911/afterSettleList.csv')
a

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,session,fbuid,FB姓名,購買權確認,商品編號,商品名稱,購買限制,留言購買數,結算確認購買數量,優惠贈送量,扣除贈送量後計價數,商品單價,商品總價,備註
0,20190911,100000090961284,洪思群,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
1,20190911,643760847,Bai Xiao Bai,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
2,20190911,1520874579,Frank Fang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
3,20190911,580856401,Herb Chang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
4,20190911,100001106450487,曾彥文,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
5,20190911,100007086393208,Da Chen Pang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
6,20190911,100002296332595,Carlos Hsieh,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,1,1,0,1,950,950,已確認
7,20190911,100000090961284,洪思群,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,1,1,0,1,950,950,已確認
8,20190911,100000138424633,YH Chang,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,2,2,0,2,950,1900,已確認
9,20190911,100004003127767,蔡宗霖,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,2,2,0,2,950,1900,已確認


# 結算金額資訊

In [12]:
def feeList(session,method='append'):

    cursor=db.cursor()
    sql="DELETE FROM FEELIST WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    df=pd.read_sql('select * from calDeliverCostFinal where session='+str(session),con=engine,index_col=['session','fbuid'])
    #製作訂單編號
    df['index']=(df['index']+1).apply(lambda s:'000'+str(s) if s<10 else '00'+str(s) if s<100  else '0'+str(s) if s<1000 else str(s))
    df['訂單編號']=str(session)+df['index']
    df=df.loc[:,['訂單編號','orderCount','deilverFreeAll','deilverFreeAllbottle','discountAll','selfGetCost','diliverCostAfterDiscountAll',
                'diliverCostAfterDeilverFreeAll','totalPriceIncludeDC']]

    df['deilverFreeAll']=df['deilverFreeAll'].apply(lambda s:"滿"+str(s)+'元免運' if s>0 else '無')
    df['deilverFreeAllbottle']=df['deilverFreeAllbottle'].apply(lambda s:"滿"+str(s)+'件免運' if s>0 else '無')
    df['discountAll']=df['discountAll'].apply(lambda s:"全團"+str(s*10)+'折優惠' if s!=1 else '無全團折扣優惠')

    df=df.rename(columns={'orderCount':'結算確認購買數量','deilverFreeAll':'金額達標免運優惠','deilverFreeAllbottle':'瓶數達標免運優惠',
                          'discountAll':'全團折扣優惠','selfGetCost':'折扣前價格','diliverCostAfterDiscountAll':'折扣後自取價',
                          'diliverCostAfterDeilverFreeAll':'運費','totalPriceIncludeDC':'含運費總價'})
    df=df.reset_index()
    df.to_sql(name='feeList', con=engine, if_exists=method)

    return df.sort_index()

a=feeList(session)
a.to_csv('/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/20190911/feeList.csv')
a

,session,fbuid,訂單編號,結算確認購買數量,金額達標免運優惠,瓶數達標免運優惠,全團折扣優惠,折扣前價格,折扣後自取價,運費,含運費總價
0,20190911,28121540,201909110001,1,無,無,無全團折扣優惠,580,580,160,740
1,20190911,36914827,201909110002,11,無,無,無全團折扣優惠,13640,13640,290,13930
2,20190911,518125422,201909110003,1,無,無,無全團折扣優惠,750,750,160,910
3,20190911,519998038,201909110004,1,無,無,無全團折扣優惠,1080,1080,160,1240
4,20190911,524402757,201909110005,1,無,無,無全團折扣優惠,1680,1680,160,1840
5,20190911,525966570,201909110006,1,無,無,無全團折扣優惠,1690,1690,160,1850
6,20190911,526497993,201909110007,2,無,無,無全團折扣優惠,1320,1320,225,1545
7,20190911,528043515,201909110008,1,無,無,無全團折扣優惠,950,950,160,1110
8,20190911,532934982,201909110009,1,無,無,無全團折扣優惠,750,750,160,910
9,20190911,544413512,201909110010,4,無,無,無全團折扣優惠,3780,3780,225,4005


#  結算後大表單輸出

In [16]:
def afterSettleBigTable(session):

    cursor=db.cursor()
    sql="DELETE FROM AFTERSETTLEBIGTABLE WHERE SESSION="+str(session)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        
    #讀取資料
    df=pd.read_sql('select * from feeList where session='+str(session),con=engine,index_col=['fbuid'])
    df2=pd.read_sql('select * from afterSettleList where session='+str(session),con=engine,index_col=['fbuid'])
    helper=pd.read_sql('select * from memberHelper',con=engine,index_col=['FbId'])

    df2=df2[df2['商品總價']!=0]
    df2['訂單編號']=df['訂單編號']
    df2=df2.loc[:,['session','訂單編號','FB姓名','商品編號','商品名稱','商品單價','留言購買數','結算確認購買數量','商品總價','備註']]

    #結算金額對齊購買酒款清單欄位
    dfReset=df.loc[:,['session','訂單編號','結算確認購買數量','折扣後自取價','運費','含運費總價']]
    dfReset['FB姓名']='總計款數'
    dfReset['商品名稱']='自取付款金額'
    dfReset['留言購買數']='運費'
    dfReset['商品總價']='宅配付款金額'
    dfReset=dfReset.rename(columns={'結算確認購買數量':'商品編號','折扣後自取價':'商品單價','運費':'結算確認購買數量','含運費總價':'備註'})
    dfReset=dfReset.loc[:,df2.columns]

    #合併資料,加入小幫手資訊
    fs=pd.concat([df2,dfReset])
    fs['小幫手']=helper['helper']
    fs['小幫手']=fs['小幫手'].fillna('佩玄')
    fs=fs.reset_index()

    fs=fs.set_index('小幫手')
    fs=fs.sort_values(['fbuid','訂單編號'])
    fs=fs.reset_index()
    fs['訂單編號']=fs['訂單編號'].astype(str)
    fs.to_sql(name='afterSettleBigTable', con=engine, if_exists='replace')
    return fs

df=afterSettleBigTable(session)
# df=df[df['小幫手'].isna()]
df.to_csv('/Users/benbilly3/Desktop/Innopro-master/Innopro_crawl_data/20190911/結算後大表單.csv')


# 資料檢視

In [ ]:
# session='201908'
# df=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['wineID'])
# s=df[df['getSure']=='sorry']
# s
# for a,b in zip([0,1,2,3],[0]):
#     print(a,b)

a="0050"
a

In [14]:
df

,小幫手,fbuid,session,訂單編號,FB姓名,商品編號,商品名稱,商品單價,留言購買數,結算確認購買數量,商品總價,備註
0,佩玄,28121540,20190911,201909110001,Jeremie Pan,4298,Mauro Molino 2018 Roero Arneis DOCG 干白酒,580,1,1,580,已確認
1,佩玄,28121540,20190911,201909110001,總計款數,1,自取付款金額,580,運費,160,宅配付款金額,740
2,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4592,"Marc Colin 2016 Chassagne-Montrachet ""Margaux"" 白",1780,1,1,1780,已確認
3,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4520,Ayala 2008 Rosé No.8 Brut Champagne,1780,1,1,1780,限加1
4,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4300,Mauro Molino 2015 Barolo Gallinotto紅,1280,2,2,2560,已確認
5,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4529,Jean-Paul & Benoit Droin 2017 Chablis 白,910,1,1,910,已確認
6,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4540,Sylvain Cholet 2016 Puligny Montrachet “Les En...,1260,1,1,1260,已確認
7,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4665,Jean Fournier 2017 Marsannay Les Longeroies紅,1180,1,1,1180,已確認
8,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4535,Digioia Royer 2015 Nuits Saint Georges “Les Ch...,1280,1,1,1280,已確認
9,佩玄,36914827,20190911,201909110002,Vincent Tk Kuo,4506,G. F. Duntze NV 'Legende' Blanc de Blancs 1er ...,990,1,1,990,已確認
